In [25]:
import utils, shapely_utils as shu, python_utils as pu
import bezier
import numpy as np
from shapely import LineString, MultiLineString, GeometryCollection
import shapely as sh

In [ ]:
# pip install shapely

In [5]:
# imports
from pathlib import Path, PurePath
import sys  

In [245]:

def bball_net(radius,m=0.15,n=0.25,total_slices=12,len_straight=4, prop_h=0.5):
    curve = bezier.Curve(np.asfortranarray(np.matrix([[0,0],[n/4,m],[n,prop_h*m]]).transpose()), degree=2)
    line = LineString([[-n/3,-n/len_straight],[0,0]])
    # return line
    l = sh.line_merge(MultiLineString([line,shu.curve_to_ls(curve)]))
    # return line
    a,b,c,d = l.bounds
    x_dist = c-a
    y_dist = d-b
    w_line,h_line = shu.size(line)
    
    p = sh.line_merge(MultiLineString([l,sh.affinity.translate(sh.affinity.scale(l, -1, 1),x_dist)]))
    # return l
    # return p
    w,h = shu.size(p)
    scale_prop = radius/w
    p = sh.affinity.scale(p, scale_prop, scale_prop)
    p2 = sh.affinity.translate(sh.affinity.scale(p, 1, -1),0,- h * scale_prop + h_line*scale_prop*2)
    lip = GeometryCollection([p,p2])
    # return lip
    # return lip
    # return lip
    tots_lips = []
    for i in range(int(total_slices/2)):
        tots_lips.append(sh.affinity.rotate(lip, i*(360/total_slices)))
    # return tots_lips
    # print(len(tots_lips))
    return GeometryCollection(tots_lips)

In [ ]:
upB = bball_net(500, 0.15,0.25,12,7,0.85)
upB

In [249]:
downB = shu.copy(upB)

In [ ]:
d = shu.size(upB)[0]
square = sh.affinity.translate(shu.Rect(upB.centroid,d/2,d/2).union(shu.Rect(upB.centroid,-d/2,d/2)),0,-d/2)
upB_half = shu.intersection_collection(upB, square)
upB_half

In [ ]:
# from shapely.ops import split
# list(split(line, a).geoms)

In [102]:
points = l.intersection(GeometryCollection(list(upB.geoms)[i+1:]))
# line = break_ls(l, l.centroid)

In [ ]:

square = sh.affinity.translate(shu.Rect(upB.centroid,d/2,d/2).union(shu.Rect(upB.centroid,-d/2,d/2)),0,-d/2)
upB_half = shu.intersection_collection(upB, square)
downB = upB.copy()
square2 = sh.affinity.translate(shu.Rect(downB.centroid,d/2,d/2).union(shu.Rect(downB.centroid,-d/2,d/2)),0,0)
downB_half = shu.intersection_collection(downB, square2)
# crec k vull fer que totes les geometries .rotate .translate .affine_transform existeixin
bottom = sh.affinity.translate(downB_half, 0, -shu.size(downB_half)[1]*3)
GeometryCollection([bottom, upB_half])
all_lines = []
for i,ls in enumerate(bottom.geoms):
    print(upB_half.geoms)
    # print(upB_half.geoms[i].coords[0],ls[0])

In [ ]:
GeometryCollection([shu.Circle(upB.centroid, d/2)]+list(upB.geoms))

In [ ]:
pu.reload(shu)
shu.join(upB,sh.affinity.translate(upB, 0, d*1.5))

In [16]:
v = LineString([[1,0],[1,2]])
hs = [{"line": LineString([[0,(0.5+i)*v.length/5],[2,(0.5+i)*v.length/5]]), "orientation": ((i%2==0)*2)-1} for i in range(5)]

In [ ]:
for h in hs:
    intersection = h['line'].intersection(v)
    h['intersection'] = intersection
hs.sort(key=lambda a: sh.distance(a['intersection'],v.boundary.geoms[0]))
hs

In [ ]:

import numpy as np
current_inline = v.boundary.geoms[0]
curves = []
intensity = 0.5
for i,h in enumerate(hs):
    if (len(hs)>i+1):
        print(h['intersection'],hs[i+1]['intersection'])
        next_inline = LineString([h['intersection'],hs[i+1]['intersection']]).centroid
    else:
        next_inline = v.boundary.geoms[1]
    line = h['line']
    point = h['intersection']

    middle_node = shu.move_point_along_line(point, line, intensity*h['orientation'])
    
    nodes = [list(a) for a in [current_inline.coords[0], middle_node.coords[0], next_inline.coords[0]]]
    # print(nodes)
    # sino transposes es pensa que ed 3d bezier
    curve = bezier.Curve.from_nodes(np.asfortranarray(np.transpose(nodes)))
    # enllaçar curves?
    curves.append(curve)
    current_inline = next_inline

In [ ]:
MultiLineString([shu.curve_to_ls(c) for c in curves]+[v]+[h['line'] for h in hs])